In [1]:
import pandas as pd
import mysql.connector


#tables creation in vaccination_db
#performed in SQL
 # Table 1: Coverage Data
    
    CREATE TABLE Coverage (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        YEAR INT,
        ANTIGEN VARCHAR(255),
        ANTIGEN_DESCRIPTION TEXT,
        COVERAGE_CATEGORY VARCHAR(255),
        COVERAGE_CATEGORY_DESCRIPTION TEXT,
        TARGET_NUMBER BIGINT,
        DOSES INT,
        COVERAGE FLOAT
    );
    
# Table 2: Incidence Rate
   
    CREATE TABLE IncidenceRate (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        YEAR INT,
        DISEASE VARCHAR(255),
        DISEASE_DESCRIPTION TEXT,
        DENOMINATOR VARCHAR(255),
        INCIDENCE_RATE FLOAT
    );
   
# Table 3: Reported Cases
    
    CREATE TABLE ReportedCases (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        YEAR INT,
        DISEASE VARCHAR(255),
        DISEASE_DESCRIPTION TEXT,
        CASES INT
    );
    

# Table 4: Vaccine Introduction
    
    CREATE TABLE VaccineIntroduction (
        ISO_3_CODE VARCHAR(100),
        COUNTRYNAME VARCHAR(255),
        WHO_REGION VARCHAR(255),
        YEAR INT,
        DESCRIPTION TEXT,
        INTRO VARCHAR(100)
    );
    

 # Table 5: Vaccine Schedule
   
    CREATE TABLE VaccineSchedule (
        ISO_3_CODE VARCHAR(100),
        COUNTRYNAME VARCHAR(255),
        WHO_REGION VARCHAR(255),
        YEAR INT,
        VACCINECODE VARCHAR(255),
        VACCINE_DESCRIPTION TEXT,
        SCHEDULEROUNDS INT,
        TARGETPOP VARCHAR(255),
        TARGETPOP_DESCRIPTION TEXT,
        GEOAREA VARCHAR(100),
        AGEADMINISTERED VARCHAR(255),
        SOURCECOMMENT TEXT
    );


In [ ]:
import pandas as pd
import mysql.connector

# Database connection parameters
host = 'localhost'
user = 'root'
password = '*****'
database = 'vaccination_db'

def import_data(file_path, table_name):
    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        cursor = conn.cursor()

        # Read the Excel file into a DataFrame
        df = pd.read_excel(file_path)

        # Replace 'inf' and 'NaN' values with None (NULL in MySQL)
        df.replace([float('inf'), float('-inf'), 'inf', 'NaN', None], None, inplace=True)

        # Generate column names from the dataframe and escape them with backticks
        columns = ', '.join([f"`{col}`" for col in df.columns])

        # Prepare the SQL query template
        for index, row in df.iterrows():
            # Convert the row to a tuple
            values = tuple(row)

            # Build the SQL query with placeholders
            sql = f"INSERT INTO {table_name} ({columns}) VALUES ({', '.join(['%s'] * len(values))})"
            
            # Execute the query
            cursor.execute(sql, values)

        # Commit the transaction
        conn.commit()
        print(f"Data imported successfully from {file_path} to {table_name}")

    except Exception as e:
        print(f"Error importing data from {file_path}: {e}")

    finally:
        # Close the connection
        conn.close()

# Example usage with Excel files
import_data('cleaned_df_coverage.xlsx', 'Coverage')
import_data('cleaned_df_incidence.xlsx', 'IncidenceRate')
import_data('cleaned_df_reported.xlsx', 'ReportedCases')
import_data('cleaned_df_vacci_intro.xlsx', 'VaccineIntroduction')
import_data('cleaned_df_vacci_schedule.xlsx', 'VaccineSchedule')

Data imported successfully from cleaned_df_vacci_schedule.xlsx to VaccineSchedule


In [ ]:
#Creating seperate Tables for diseases, vaccines, countries and populate data
host = "localhost"
user = "root"
password = "prijags20"
database = "vaccination_db"

try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()


# Creating Countries table
    create_countries_table = """
    CREATE TABLE Countries (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        PRIMARY KEY (CODE) 
    );
    """
    cursor.execute(create_countries_table)
     # Insert unique country data from Coverage table
    insert_countries_from_coverage = """
    INSERT IGNORE INTO Countries (`GROUP`, CODE, NAME)
    SELECT DISTINCT `GROUP`, CODE, NAME 
    FROM Coverage;
    """
    cursor.execute(insert_countries_from_coverage)

    # Create Years table
    create_years_table = """
    CREATE TABLE Years (
        YEAR INT PRIMARY KEY
    );
    """
    cursor.execute(create_years_table)

    # Insert unique years from vaccineintroduction table
    insert_years_from_vaccineintroduction = """
    INSERT IGNORE INTO Years (YEAR)
    SELECT DISTINCT YEAR 
    FROM VaccineIntroduction;
    """
    cursor.execute(insert_years_from_vaccineintroduction)
    # Create Diseases table
    create_diseases_table = """
    CREATE TABLE Diseases (
        DISEASE VARCHAR(255) PRIMARY KEY,
        DISEASE_DESCRIPTION TEXT
    );
    """
    cursor.execute(create_diseases_table)

    # Insert unique diseases from IncidenceRate table
    insert_diseases_from_incidencerate = """
    INSERT IGNORE INTO Diseases (DISEASE, DISEASE_DESCRIPTION)
    SELECT DISTINCT DISEASE, DISEASE_DESCRIPTION 
    FROM IncidenceRate;
    """
    cursor.execute(insert_diseases_from_incidencerate)
    # Create WHO_Region table
    create_who_region_table = """
    CREATE TABLE WHO_Region (
        ISO_3_CODE VARCHAR(100),
        COUNTRYNAME VARCHAR(255),
        WHO_REGION VARCHAR(255),
        PRIMARY KEY (ISO_3_CODE, COUNTRYNAME) 
    );
    """
    cursor.execute(create_who_region_table)

    # Insert unique WHO_Region data from VaccineSchedule table
    insert_who_region_from_VaccineSchedule = """
    INSERT IGNORE INTO WHO_Region (ISO_3_CODE, COUNTRYNAME, WHO_REGION)
    SELECT DISTINCT ISO_3_CODE, COUNTRYNAME, WHO_REGION 
    FROM VaccineSchedule;
    """
    cursor.execute(insert_who_region_from_VaccineSchedule)

 # Create Antigens table
    create_antigens_table = """
    CREATE TABLE IF NOT EXISTS Antigens (
        ANTIGEN VARCHAR(255) PRIMARY KEY,
        ANTIGEN_DESCRIPTION TEXT
    );
    """
    cursor.execute(create_antigens_table)

    # Insert unique antigens from Coverage table
    insert_antigens = """
    INSERT IGNORE INTO Antigens (ANTIGEN, ANTIGEN_DESCRIPTION)
    SELECT DISTINCT ANTIGEN, ANTIGEN_DESCRIPTION
    FROM Coverage;
    """
    cursor.execute(insert_antigens)
    
     # Create Vaccines table
    create_vaccines_table = """
    CREATE TABLE IF NOT EXISTS Vaccines (
        VACCINECODE VARCHAR(255) PRIMARY KEY,
        VACCINE_DESCRIPTION TEXT
    );
    """
    cursor.execute(create_vaccines_table)

    # Insert unique vaccines from VaccineSchedule table
    insert_vaccines = """
    INSERT IGNORE INTO Vaccines (VACCINECODE, VACCINE_DESCRIPTION)
    SELECT DISTINCT VACCINECODE, VACCINE_DESCRIPTION
    FROM VaccineSchedule;
    """
    cursor.execute(insert_vaccines)
    
    conn.commit()
    print("Tables are created and data is inserted")
except mysql.connector.Error as e:
    print(f"Database error: {e}")

finally:
    conn.close()


Tables are created and data is inserted


In [ ]:
#adding foreing key constraints to all tables
# Database connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='prijags20',
    database='vaccination_db'
)

cursor = conn.cursor()

try:
    # Coverage Table
    cursor.execute("""
        ALTER TABLE coverage
        ADD CONSTRAINT fk_coverage_country FOREIGN KEY (CODE) REFERENCES countries(CODE),
        ADD CONSTRAINT fk_coverage_year FOREIGN KEY (YEAR) REFERENCES years(YEAR),
        ADD CONSTRAINT fk_coverage_antigen FOREIGN KEY (ANTIGEN) REFERENCES antigens(ANTIGEN);
    """)

    # IncidenceRate Table
    cursor.execute("""
        ALTER TABLE incidencerate
        ADD CONSTRAINT fk_incidence_country FOREIGN KEY (CODE) REFERENCES countries(CODE),
        ADD CONSTRAINT fk_incidence_year FOREIGN KEY (YEAR) REFERENCES years(YEAR),
        ADD CONSTRAINT fk_incidence_disease FOREIGN KEY (DISEASE) REFERENCES diseases(DISEASE);
    """)

    # ReportedCases Table
    cursor.execute("""
        ALTER TABLE reportedcases
        ADD CONSTRAINT fk_reported_country FOREIGN KEY (CODE) REFERENCES countries(CODE),
        ADD CONSTRAINT fk_reported_year FOREIGN KEY (YEAR) REFERENCES years(YEAR),
        ADD CONSTRAINT fk_reported_disease FOREIGN KEY (DISEASE) REFERENCES diseases(DISEASE);
    """)

    # VaccineIntroduction Table
    cursor.execute("""
        ALTER TABLE vaccineintroduction
        ADD CONSTRAINT fk_intro_year FOREIGN KEY (YEAR) REFERENCES years(YEAR),
        ADD CONSTRAINT fk_intro_country FOREIGN KEY (ISO_3_CODE) REFERENCES who_region(ISO_3_CODE);
    """)

    # VaccineSchedule Table
    cursor.execute("""
        ALTER TABLE vaccineschedule
        ADD CONSTRAINT fk_schedule_year FOREIGN KEY (YEAR) REFERENCES years(YEAR),
        ADD CONSTRAINT fk_schedule_vaccine FOREIGN KEY (VACCINECODE) REFERENCES vaccines(VACCINECODE),
        ADD CONSTRAINT fk_schedule_country FOREIGN KEY (ISO_3_CODE) REFERENCES who_region(ISO_3_CODE);
    """)

    conn.commit()
    print("Foreign keys added successfully.")

except mysql.connector.MySQLError as e:
    print(f"Error adding foreign keys: {e}")

finally:
    cursor.close()
    conn.close()

Foreign keys added successfully.


#Dropping redundant data from tables
#performed in SQL

ALTER TABLE coverage
DROP COLUMN NAME,
DROP COLUMN `GROUP`,
DROP COLUMN ANTIGEN_DESCRIPTION;

ALTER TABLE incidencerate
DROP COLUMN NAME,
DROP COLUMN `GROUP`,
DROP COLUMN DISEASE_DESCRIPTION;

ALTER TABLE reportedcases
DROP COLUMN NAME,
DROP COLUMN `GROUP`,
DROP COLUMN DISEASE_DESCRIPTION;

ALTER TABLE vaccineintroduction
DROP COLUMN COUNTRYNAME,
DROP COLUMN WHO_REGION;

ALTER TABLE vaccineschedule
DROP COLUMN COUNTRYNAME,
DROP COLUMN WHO_REGION,
DROP COLUMN VACCINE_DESCRIPTION;